In [11]:
import pandas as pd
import requests

from datetime import datetime 

import time 


In [2]:
from subgraphkit import SubgraphKit

sgk = SubgraphKit()

In [3]:
# -- list of dexs 
deployments = sgk.deployments
# -- filter df for exchanges on ethereum
dex_eth = deployments[ (deployments['subgraph_type'] == 'exchanges') & (deployments['network'] == 'ethereum') ]
dex_eth

,subgraph_type,network,protocol,deployment,url_slug,url
3,exchanges,ethereum,balancer-v2,balancer-v2-ethereum,balancer-v2-ethereum,https://api.thegraph.com/subgraphs/name/messar...
5,exchanges,ethereum,bancor-v3,bancor-v3-ethereum,bancor-v3-ethereum,https://api.thegraph.com/subgraphs/name/messar...
12,exchanges,ethereum,curve,curve-ethereum,curve-finance-ethereum,https://api.thegraph.com/subgraphs/name/messar...
23,exchanges,ethereum,saddle-finance,saddle-finance-ethereum,saddle-finance-ethereum,https://api.thegraph.com/subgraphs/name/messar...
36,exchanges,ethereum,sushiswap,sushiswap-ethereum,sushiswap-ethereum,https://api.thegraph.com/subgraphs/name/messar...
42,exchanges,ethereum,uniswap-v2,uniswap-v2-ethereum,uniswap-v2-ethereum,https://api.thegraph.com/subgraphs/name/messar...
44,exchanges,ethereum,uniswap-v3,uniswap-v3-ethereum,uniswap-v3-ethereum,https://api.thegraph.com/subgraphs/name/messar...


In [4]:
# -- function to get all swaps for a dex
def get_swaps(dex_slug, timestamp):
  # -- get dex object 
  dex = sgk.get_subgraph(dex_slug)

  # -- get swaps 
  dex.swaps.filter(timestamp__lt= timestamp ).order_by('-timestamp').select_fields('timestamp', 'id', 'hash', 'blockNumber', 'to', 'from', 'token_in__symbol', 'token_out__symbol', 'amountIn', 'amountOut', 'amountInUSD', 'amountOutUSD', 'pool_id').all()

  # -- get last timestamoe 
  last_ts = dex.df['timestamp'].iloc[-1]

  return dex.df, last_ts

In [20]:
def get_all_swaps(dex_slug, after_timestamp = 0, folderpath = '', before_timestamp = 0):
  # -- get current timestamp
  if before_timestamp == 0:
    current_ts = int(time.time())
  else: 
    current_ts = before_timestamp

  # -- get first set of swaps 
  df, last_ts = get_swaps(dex_slug, current_ts)

  # -- convert timestamp to datetime
  dt = datetime.fromtimestamp(int(time.time())).strftime('%Y-%m-%d')

  # -- add protocol columns 
  dex_info = dex_slug.rsplit('-', 1)

  # -- create protocol and chain columns
  df['protocol'] = dex_info[0]
  df['chain'] = dex_info[1]

  # -- save to csv
  df.to_csv(folderpath + dex_slug + '_'+ dt + '.csv', index=False)
  del df 

  # -- loop until we reach the after_timestamp
  while last_ts > after_timestamp:
    # -- get swaps
    df, last_ts = get_swaps(dex_slug, last_ts)

    # -- convert timestamp to datetime
    dt = datetime.fromtimestamp(int(time.time())).strftime('%Y-%m-%d')

    # -- add protocol columns 
    dex_info = dex_slug.rsplit('-', 1)

    # -- create protocol and chain columns
    df['protocol'] = dex_info[0]
    df['chain'] = dex_info[1]

    # -- save to csv
    df.to_csv(folderpath + dex_slug + '_' + dt + '.csv', index=False)

    del df 

    # -- break loop if len less than million
    if len(df) < 1000000:
      break


In [24]:
list(dex_eth['deployment'])

['balancer-v2-ethereum',
 'bancor-v3-ethereum',
 'curve-ethereum',
 'saddle-finance-ethereum',
 'sushiswap-ethereum',
 'uniswap-v2-ethereum',
 'uniswap-v3-ethereum']

In [27]:
dexs = list(dex_eth['deployment'])
dexs.remove('saddle-finance-ethereum')

In [28]:
# -- get all swaps for all dexs
for dex in reversed(dexs):
  print(dex)
  if dex == 'uniswap-v3-ethereum':
    get_all_swaps(dex, folderpath = 'eth-swaps/', before_timestamp = 1663337267) # -- hardcode bc did as test first 
  else: 
    get_all_swaps(dex, folderpath = 'eth-swaps/')

uniswap-v3-ethereum
uniswap-v2-ethereum
sushiswap-ethereum
curve-ethereum
bancor-v3-ethereum
balancer-v2-ethereum
